In [2]:
import pandas as pd
import numpy as np
Y=pd.read_csv('Phenotype.csv').astype(np.int8)
print(Y.shape)

(1000, 1)


In [3]:
type(Y["Phenotype"][2])

numpy.int8

In [6]:
X=pd.read_csv("Genotype.csv")

MemoryError: 

In [4]:
X=pd.DataFrame()

In [5]:
chunks = []
df_chunks = pd.read_csv("Genotype.csv", chunksize=200)
for chunk in df_chunks:
    chunks.append(chunk)
X = pd.concat(chunks, ignore_index=True)

MemoryError: 

In [ ]:
X = X.sample(n=200000, axis=1, random_state=42).astype(np.int8)

In [ ]:
missing_values=X.isna().sum().sum()
print(missing_values)

In [ ]:
# Minor Allele Frequency Filtering. Threshold=0.05
def calculate_maf(G):
    allele_counts = G.value_counts()
    p = (2*allele_counts.get(0,0)+allele_counts.get(1,0))/(2*len(G))
    return min(p, 1-p)

maf_threshold = 0.05
maf = X.apply(calculate_maf, axis=0)             #calculate MAF of each SNP
filtered_snps = maf[maf >= maf_threshold].index
X = X[filtered_snps]

In [ ]:
X.shape

In [ ]:
# HWE Filtering of Controls:
from scipy.stats import chi2_contingency
def HWE_test(G):
    obs_AA = (G==0).sum()
    obs_Aa = (G==1).sum()
    obs_aa = (G==2).sum()
    p = (obs_AA+2*obs_Aa)/(2*len(G))
    q = 1-p
    obs=[obs_AA, obs_Aa, obs_aa]
    exp_AA = p**2
    exp_Aa = 2*p*q
    exp_aa = q**2
    exp = [exp_AA, exp_Aa, exp_aa]
    cont_table = np.array([obs,exp])
    p_value = chi2_contingency(cont_table)[1]
    return p_value

control_ind = Y[Y['Phenotype']==0].index        # find out the incdices of controls.
HWE_p_values = X.loc[control_ind].apply( HWE_test, axis=0)            # a series with p-values of all snps, but counted only on control group
filtered_SNP_indices = HWE_p_values[HWE_p_values >= 0.001].index
X=X[filtered_SNP_indices]

In [ ]:
X

In [ ]:
print(X.shape, type(X.iloc[1,1]))

In [ ]:
# LD Pruning:
from scipy.stats import pearsonr
import numpy as np
def ld_prune(genotype_df, ld_threshold):
    pruned_snps = []
    removed_snps = set()
    
    for i, snp1 in enumerate(genotype_df.columns):
        if snp1 in removed_snps:
            continue
        
        pruned_snps.append(snp1)
        snp1_genotypes = genotype_df[snp1]
        
        for snp2 in genotype_df.columns[i+1:]:
            snp2_genotypes = genotype_df[snp2]
            r, _ = pearsonr(snp1_genotypes, snp2_genotypes)
            r_squared = r ** 2
            
            if r_squared > ld_threshold:
                removed_snps.add(snp2)
    
    return genotype_df[pruned_snps]

X = ld_prune(X, ld_threshold=0.2)
'''def ld_prune(genotype_df, ld_threshold):
    # Calculate correlation matrix using vectorized operations
    corr_matrix = np.corrcoef(genotype_df.T)

    # Find SNPs with pairwise correlation above the threshold
    pairs = np.where(corr_matrix > ld_threshold)
    pairs = np.array(list(zip(*pairs)))

    # Remove SNPs with high correlation
    removed_snps = set()
    for snp1, snp2 in pairs:
        if snp1 not in removed_snps and snp2 not in removed_snps:
            removed_snps.add(snp2)

    # Return pruned genotype data
    return genotype_df.drop(columns=removed_snps)

X_reduced = ld_prune(X_reduced, ld_threshold=0.2)'''

In [ ]:
X

In [ ]:
import pandas as pd
from sklearn.linear_model import LogisticRegression

def perform_gwas(genotype_df, phenotype_df):
    
    results = []
    for snp in genotype_df.columns[0:]:
        x = genotype_df[[snp]]
        y = phenotype_df['phenotype']

        # Fit logistic regression model
        model = LogisticRegression()
        model.fit(x, y)

        # Extract coefficient and p-value
        coef = model.coef_[0][0]
        p_value = model.score(x, y) 

        results.append((snp, coef, p_value))

    # Create DataFrame with results
    gwas_results = pd.DataFrame(results, columns=['SNP', 'Coefficient', 'P-Value'])

   

    return gwas_results

gwas_results = perform_gwas(X, Y)

# Filter significant SNPs based on adjusted p-value threshold
significant_snps = gwas_results[gwas_results['Adjusted P-Value'] < 0.05]
print(significant_snps)